In [1]:
from app.config import get_settings
import os

settings = get_settings()

In [4]:
from app.knowledge.infra.repository.embeddings_repo import EmbeddingsRepository
from app.knowledge.application.embeddings_service import EmbeddingsService
from app.knowledge.infra.db_models.embeddings import Embeddings
from app.knowledge.infra.db_models.weekly_report import WeeklyReport
from app.utils.db_utils import row_to_dict
from app.database import SessionLocal
from google import genai
from google.genai.types import EmbedContentConfig
from app.utils.id_utils import generate_nanoid

embeddings_repo = EmbeddingsRepository()
embeddings_service = EmbeddingsService(embeddings_repo=embeddings_repo)

with SessionLocal() as db:
    rows = db.query(WeeklyReport).all()

data = [row_to_dict(row) for row in rows]

for row in data:
    content = f"<document>\ntitle:{row["start_date"]} ~ {row["end_date"]} weekly report\ncontent:{row['content']}\n</document>"
    contents = [row["content"]]

    client = genai.Client(vertexai=True, project=settings.GOOGLE_CLOUD_PROJECT, location=settings.GOOGLE_CLOUD_LOCATION)
    response = client.models.embed_content(
        model="gemini-embedding-001",
        contents=contents,
        config=EmbedContentConfig(
            task_type="RETRIEVAL_DOCUMENT",  # Optional
        ),
    )
    embedding = response.embeddings[0].values
    print(len(embedding))
    break

    # new_data = Embeddings(
    #     id=generate_nanoid(),
    #     date=row["end_date"],
    #     origin_id=row["id"],
    #     origin_type="weekly_report",
    #     content=row["content"],
    #     embedding=embedding
    # )

    # embeddings_service.create(new_data)

768
